In [1]:
cd ./American_dad_script/

C:\Users\berte\Desktop\M2_AIC\NLP\Projet\American_dad_script


In [4]:
# Load Larger LSTM network and generate text
import sys
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import np_utils
import os
# load ascii text and covert to lowercase

full_text = []

for filename in os.listdir(os.getcwd()):
    if (filename!='.ipynb_checkpoints'):
        raw_text = open(filename, 'r', encoding='utf-8').read()
        full_text.append(raw_text.lower())
flat_list = (', '.join(full_text))

flat_list = flat_list[:50000].rstrip()

In [5]:
whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZ')
answer = ''.join(filter(whitelist.__contains__, flat_list))


In [6]:
# create mapping of unique chars to integers
chars = sorted(list(set(answer)))
char_to_int = dict((c, i) for i, c in enumerate(chars))

In [7]:
print(char_to_int)

{' ': 0, 'a': 1, 'b': 2, 'c': 3, 'd': 4, 'e': 5, 'f': 6, 'g': 7, 'h': 8, 'i': 9, 'j': 10, 'k': 11, 'l': 12, 'm': 13, 'n': 14, 'o': 15, 'p': 16, 'q': 17, 'r': 18, 's': 19, 't': 20, 'u': 21, 'v': 22, 'w': 23, 'x': 24, 'y': 25, 'z': 26}


In [8]:
# summarize the loaded data
n_chars = len(answer)
n_vocab = len(chars)
print( "Total Characters: ", n_chars)
print( "Total Vocab: ", n_vocab)
# prepare the dataset of input to output pairs encoded as integers
seq_length = 100
dataX = []
dataY = []
for i in range(0, n_chars - seq_length, 1):
	seq_in = answer[i:i + seq_length]
	seq_out = answer[i + seq_length]
	dataX.append([char_to_int[char] for char in seq_in])
	dataY.append(char_to_int[seq_out])
n_patterns = len(dataX)
print ("Total Patterns: ", n_patterns)
# reshape X to be [samples, time steps, features]
X = numpy.reshape(dataX, (n_patterns, seq_length, 1))
# normalize
X = X / float(n_vocab)
# one hot encode the output variable
y = np_utils.to_categorical(dataY)
# define the LSTM model
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(256))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="../WEIGHT_NLP/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model

Total Characters:  46498
Total Vocab:  27
Total Patterns:  46398


In [15]:
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)


Epoch 1/50
46398/46398 [==============================] - 1199s 26ms/step - loss: 2.8172

Epoch 00001: loss improved from inf to 2.81716, saving model to ../WEIGHT_NLP/weights-improvement-01-2.8172-bigger.hdf5


OSError: Unable to create file (unable to open file: name = '../WEIGHT_NLP/weights-improvement-01-2.8172-bigger.hdf5', errno = 2, error message = 'No such file or directory', flags = 13, o_flags = 302)

In [13]:
ls

 Le volume dans le lecteur C s'appelle Windows
 Le num‚ro de s‚rie du volume est 28E6-BFFF

 R‚pertoire de C:\Users\berte\Desktop\M2_AIC\NLP\Projet

05/11/2019  13:53    <DIR>          .
05/11/2019  13:53    <DIR>          ..
05/11/2019  13:52    <DIR>          .ipynb_checkpoints
05/11/2019  13:52    <DIR>          American_dad_script
05/11/2019  13:53            30ÿ229 NLP.ipynb
28/10/2019  19:35            18ÿ490 projet.ipynb
25/10/2019  14:28             1ÿ678 scrapping.ipynb
28/10/2019  19:30    <DIR>          TEST
28/10/2019  19:28             5ÿ775 test.ipynb
05/11/2019  13:50    <DIR>          WEIGHT_NLP
               4 fichier(s)           56ÿ172 octets
               6 R‚p(s)  140ÿ804ÿ853ÿ760 octets libres


In [14]:
import sys
...
# load the network weights
filename = "./WEIGHT_NLP/weights-improvement-01-2.4640-bigger.hdf5"
model.load_weights(filename)
model.compile(loss='categorical_crossentropy', optimizer='adam')
# define the checkpoint
filepath="../WEIGHT_NLP/weights-improvement-{epoch:02d}-{loss:.4f}-bigger.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]
# fit the model
model.fit(X, y, epochs=50, batch_size=64, callbacks=callbacks_list)

OSError: Unable to open file (file signature not found)

In [49]:
int_to_char = dict((i, c) for i, c in enumerate(chars))
...
# pick a random seed
start = numpy.random.randint(0, len(dataX)-1)
pattern = dataX[start]
print ("Seed:")
print ("\"", ''.join([int_to_char[value] for value in pattern]), "\"")
# generate characters
for i in range(100):
    x = numpy.reshape(pattern, (1, len(pattern), 1))
    x = x / float(n_vocab)
    prediction = model.predict(x, verbose=0)
    index = numpy.argmax(prediction)
    result = int_to_char[index]
    seq_in = [int_to_char[value] for value in pattern]
    sys.stdout.write(result)
    pattern.append(index)
    pattern = pattern[1:len(pattern)]
print ("\nDone.")

Epoch 1/50
  512/45673 [..............................] - ETA: 9:31 - loss: 2.4728 

KeyboardInterrupt: 